In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

c:\Users\USER\Downloads\machine learning\project\Regression_ML_EndtoEnd\.venv\Scripts\python.exe
3.0.4
c:\Users\USER\Downloads\machine learning\project\Regression_ML_EndtoEnd\.venv\Lib\site-packages\xgboost\__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

c:\Users\USER\Downloads\machine learning\project\Regression_ML_EndtoEnd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv(r"C:\Users\USER\Downloads\machine learning\project\Regression_ML_EndtoEnd\data\processed\frature_enginered_train.csv")
eval_df  = pd.read_csv(r"C:\Users\USER\Downloads\machine learning\project\Regression_ML_EndtoEnd\data\processed\feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [5]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [8]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri(r"file:///C:/Users/USER/Downloads/machine learning/project/Regression_ML_EndtoEnd/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/11/09 19:20:57 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.


[I 2025-11-09 19:20:57,472] A new study created in memory with name: no-name-2ab78f5b-823c-48a1-936c-845cd75adcbd
[I 2025-11-09 19:23:01,073] Trial 0 finished with value: 75151.3863296602 and parameters: {'n_estimators': 947, 'max_depth': 7, 'learning_rate': 0.0949761003352388, 'subsample': 0.9793814643868698, 'colsample_bytree': 0.9731659511994579, 'min_child_weight': 7, 'gamma': 4.322778877770883, 'reg_alpha': 2.295725908609192, 'reg_lambda': 0.00347977132587179}. Best is trial 0 with value: 75151.3863296602.
[I 2025-11-09 19:23:37,577] Trial 1 finished with value: 73437.26764646388 and parameters: {'n_estimators': 649, 'max_depth': 5, 'learning_rate': 0.09632024535008621, 'subsample': 0.59502718602039, 'colsample_bytree': 0.7209242432258904, 'min_child_weight': 4, 'gamma': 0.1782423957795609, 'reg_alpha': 1.5209306131543984e-08, 'reg_lambda': 0.0008121273147722099}. Best is trial 1 with value: 73437.26764646388.
[I 2025-11-09 19:24:18,102] Trial 2 finished with value: 74367.58724225

Best params: {'n_estimators': 842, 'max_depth': 9, 'learning_rate': 0.013919576885127374, 'subsample': 0.503249261481169, 'colsample_bytree': 0.5075976377103861, 'min_child_weight': 9, 'gamma': 3.5586985488649376, 'reg_alpha': 0.24014819893840145, 'reg_lambda': 7.153351311440565}


In [11]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 31265.860768575887
RMSE: 67663.99188650002
R²: 0.9646185796977526


c:\Users\USER\Downloads\machine learning\project\Regression_ML_EndtoEnd\.venv\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [18:07:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/11/11 18:07:38 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/11 18:07:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
